
#FDMS tme4


TP4 : Implémentation des modèles de CF avec descente de gradient stochastique
    
    Modèle classique L2 (sans Biais)
    Modèle avec biais utilisateur et items
    Evaluation des modèles sur la base MovieLens

##Loading Movie Lens

On importe les données

In [1]:
import numpy as np
import copy
from collections import defaultdict

def loadMovieLens(path='ml-100k/'):
    #Get movie titles
    movies={}
    for line in open(path+'/u.item'):
        (id,title)=line.split('|')[0:2]
        movies[id]=title

    # Load data
    prefs={}
    for line in open(path+'/u.data'):
        (user,movieid,rating,ts)=line.split('\t')
        prefs.setdefault(user,{})
        prefs[user][movies[movieid]]=float(rating)
    return prefs


##Transform Data & split sets

On convertit les données en liste de tuples pour faciliter le split entre test et train.

On split aléatoirement sans remise avec 80% en train et 20% en test.

On transforme les ratings dans une matrice Users*Items pour pouvoir faire des calculs matriciels par la suite.



In [2]:
#now reindexing everything
#flatten the whole thing
def flatten_dict(ratings):
    tuples_items = []
    for userId, userMovies in ratings.iteritems() :
        for movieId, rating in userMovies.iteritems() :
            tuples_items.append((userId,movieId, rating))
    return np.array(tuples_items)

def split_data(a_ratings, perc):
    indexes = np.random.permutation(range(len(a_ratings)))
    seuil = perc * len(a_ratings)
    return a_ratings[indexes[:seuil],:], a_ratings[indexes[seuil:],:]
    
def filter_test_rating(train_ratings, test_ratings):
    #we want the test set to contain only films and users present in the train set
    users = {}
    items = {}
                                    
    for i in range(len(train_ratings)):
        users[train_ratings[i,0]] = True
        items[train_ratings[i,1]] = True

    filtered_test = []
    for i in range(len(test_ratings)):
        try :
            users[test_ratings[i,0]]
            items[test_ratings[i,1]]
            
            filtered_test.append(test_ratings[i])
        except KeyError:
            pass # do not add
    return np.array(filtered_test)
from scipy.sparse import dok_matrix

#build only indexes from the train ratings because we assume the test set contains the same keys...
def build_indexes(train_ratings):
    idx_u = {}
    idx_i = {}
    
    i = 0
    j = 0
    for ind_l in range(len(train_ratings)) :
        user_id, movie_id, rating = train_ratings[ind_l,:]
        try :
            idx_u[user_id]
        except KeyError :
            idx_u[user_id] = i
            i += 1
        try:
            idx_i[movie_id]
        except KeyError:
            idx_i[movie_id] = j
            j += 1
    return idx_u, idx_i
"""CSR or CSC should be your preferred option, as most functions are built for these types.
BSR is a block version of CSR. COO and DOK are convenient for data entry, but once you have it 
all entered, you'll want to convert it to one of the other types"""
def build_matrix(ratings, idx_u, idx_i):
    #build matrix
    m = np.zeros((len(idx_u), len(idx_i)))#non sparse
    for i in range(len(ratings)) :
        user_id, movie_id, rating = ratings[i,:]
        m[idx_u[user_id], idx_i[movie_id]] = rating
    return  m#spm


In [3]:
prefs = loadMovieLens()

a_ratings = flatten_dict(prefs)
train_ratings, test_ratings = split_data(a_ratings, 0.8)
filt_test_ratings = filter_test_rating(train_ratings, test_ratings)

print 'nbTrain', len(train_ratings)
print 'nbTest',len(test_ratings)
print 'nbTestFiltered',len(filt_test_ratings)


idx_u, idx_i = build_indexes(train_ratings)
print 'nbUsers',len(idx_u)
print 'nbMovies',len(idx_i)

print '----'
#il faut que les deux matrices soient construites avec les mêmes indexes...
train_m = build_matrix(train_ratings, idx_u, idx_i)
test_m = build_matrix(filt_test_ratings, idx_u, idx_i)
print 'nbRatingsTrain',np.count_nonzero(train_m)
print 'nbRatingsTestFilt',np.count_nonzero(test_m)


nbTrain 79754
nbTest 19939
nbTestFiltered 19912
nbUsers 943
nbMovies 1638
----
nbRatingsTrain 79754
nbRatingsTestFilt 19912


##Estimateurs : Baseline

Les 2 baselines testées correspondenet à la note moyenne donnée par utilisateur et par film

In [4]:
class Estimator():
    def __init__(self,idx_u, idx_i):
        self.idx_u = idx_u
        self.idx_i = idx_i
    def fit(self, m):
        pass
    def predict(self, m):
        pass
    def score(self, m):
        nzero_ind = np.nonzero(m)
        preds = self.predict(m)   
        #il est necessaire de ne regarder que les indices non egaux a zero
        #sinon on va regarder des predictions qui ne nous interessent pas...
        #on suppose dans m que les cases non à zero sont celles qui nous interessent
        #x'Est couteux en memoire mais bon... c'Est plus facile pour les operations
        dif_abs = np.power(np.subtract(m[nzero_ind],preds[nzero_ind]),2)
        return dif_abs.mean()
    
#le mean est un peu dégueu mais faut qu on le fasse sur tout ce qui est à pas zero...
class BaselineMeanUser(Estimator):
    def __init__(self, idx_u, idx_i):
        Estimator.__init__(self,idx_u, idx_i)
    def fit(self, train_m):#on suppose que les indixes des ibjets sont dans k'index avec lequel on initialise...
        self.mean_by_user = [train_m[i,:][np.nonzero(train_m[i,:])].mean() for i in range(train_m.shape[0])]
        return np.array(self.mean_by_user)
    def predict(self, test_m):
        preds = np.zeros((len(self.idx_u), len(self.idx_i)))
        preds = preds.T + self.mean_by_user #on met sur les lignes !Attenton broadcast
        return preds.T
    
class BaselineMeanMovie(Estimator):
    def __init__(self, index_users, index_items):
        Estimator.__init__(self,index_users,index_items)
    def fit(self, train_m):
        self.mean_by_movie = [train_m[:,j][np.nonzero(train_m[:,j])].mean() for j in range(train_m.shape[1])]
        return np.array(self.mean_by_movie)
    def predict(self, test_m):
        preds = np.zeros((len(self.idx_u), len(self.idx_i)))
        preds = preds + self.mean_by_movie #on met sur les colonnes
        return preds
    

In [5]:
estimateur = BaselineMeanUser(idx_u,idx_i)
estimateur.fit(train_m)
score = estimateur.score(train_m)
print 'SquaredDistance train BLMeanUser',score
score = estimateur.score(test_m)
print 'SquaredDistance test BLMeanUser',score

estimateur = BaselineMeanMovie(idx_u,idx_i)
estimateur.fit(train_m)
score = estimateur.score(train_m)
print 'SquaredDistance train BLMeanMovie',score
score = estimateur.score(test_m)
print 'SquaredDistance test BLMeanMovie',score

SquaredDistance train BLMeanUser 1.05847516135
SquaredDistance test BLMeanUser 1.09249901866
SquaredDistance train BLMeanMovie 0.994436913073
SquaredDistance test BLMeanMovie 1.05049164588


Le score calculé correspond à la moyenne des erreurs au carré.

La baseline MeanMovie est plus pertinente que la baseline MeanUser.

La distance est plus grande pour les données de test que d'entraînement.

##Modèle Classique L2 (Sans Biais)

In [6]:
class MatrixFactorizationEstimator(object):
    def __init__(self, lamb,eps=1e-4,K=3,I = 10,doprint = False,itprint = 10 ):
        self.eps = eps
        self.K = K
        self.I = I
        self.lamb = lamb
        self.itprint = itprint
        self.doprint = doprint
    def initialize(self,R):
        self.P = np.random.rand(R.shape[0],self.K)#utilisateurs
        self.Q = np.random.rand(self.K,R.shape[1]) #items
        self.log_loss=[]
        self.log_score=[]
    def fit(self, R):
        self.initialize(R)
        ind = np.nonzero(R)
        nb_ex = len(ind[0])
        for it in xrange(self.I):         
            #recupere les indices des exemples
            loss = 0.
            for stoch in xrange(nb_ex):#gradient stochastique
                rand_ind = np.random.randint(nb_ex)
                i, j = ind[0][rand_ind], ind[1][rand_ind]
               
                rij = np.dot(self.P[i,:].reshape((1, self.K)), self.Q[:,j].reshape((self.K, 1)))
                eij = R[i,j] - rij
               
                temp_P = (1 - self.lamb * self.eps) * self.P[i,:] + 2. * self.eps * eij * self.Q[:,j]#.reshape()
                self.Q[:,j] = (1 - self.lamb * self.eps) * self.Q[:,j] + 2. * self.eps * eij * self.P[i,:]#.reshape()
                self.P[i,:] = temp_P#maj simultanée
               
                loss += eij**2 #loss = self.loss(R)
               
            self.log_loss.append(loss/nb_ex)
            if it % self.itprint==0:               
                if self.doprint :
                    print '\nit = ', it
                    print 'loss : ', np.mean(self.log_loss[-self.itprint:])#acc_loss / self.itprint
                    score = self.score(R)
                    self.log_score.append(score)
                    print 'score ',score
                    print 'true loss', self.loss(R)
    def predict(self, test_m):
        return np.dot(self.P, self.Q)
    def score(self, m):
        return ((m[np.nonzero(m)] - self.predict(m)[np.nonzero(m)] )**2).mean()
    def loss(self, m):
        return ((m[np.nonzero(m)] - self.predict(m)[np.nonzero(m)] )**2).mean().sum() + self.lamb*(np.power(self.P,2).sum() + np.power(self.Q,2).sum())
        

In [8]:

outfile = "data_exp"
estimateur1 = MatrixFactorizationEstimator( lamb=0.2,eps=1e-5,K=10,I = 1200,doprint = True,itprint = 50)
estimateur1.fit(train_m)
score_train = estimateur1.score(train_m)
print 'score BIAS, train : ',score_train
score_test = estimateur1.score(test_m)
print 'score BIAS, test : ',score_test


it =  0
loss :  2.804335849
score  2.78812887946
true loss 1725.80994686

it =  50
loss :  2.11406268868
score  1.65771515719
true loss 1811.55511081

it =  100
loss :  1.44558394126
score  1.29263841292
true loss 1856.94608516

it =  150
loss :  1.21050757422
score  1.1427402866
true loss 1879.60038379

it =  200
loss :  1.09980293351
score  1.06378688521
true loss 1891.97087117

it =  250
loss :  1.03651766734
score  1.01506629228
true loss 1899.24023036

it =  300
loss :  0.996617963594
score  0.981794379517
true loss 1903.75555892

it =  350
loss :  0.969384606074
score  0.957530304814
true loss 1906.73152772

it =  400
loss :  0.948284934776
score  0.939097707057
true loss 1908.56151577

it =  450
loss :  0.932413987343
score  0.92455273674
true loss 1909.57415914

it =  500
loss :  0.918440472535
score  0.912714122974
true loss 1910.22253935

it =  550
loss :  0.907143123562
score  0.903127487072
true loss 1910.22240026

it =  600
loss :  0.899585872914
score  0.895022014217
true

L'algorithme met du temps à converger (1200 époques), sûrement du au fait que nous ayons codé l'algorithme avec des matrices normales. Pour atteindre un score un peu meilleur que les baslines, l'algorithme a tourné à peu près 20 mn.

On observe un surapprentissage assez marqué  dans la différence de score entre train et test.

##Modèle classique L2  (Avec biais utilisateur et film)  

Dans ce modèle on rajoute un biais moyen et un biais utilisateur et film.

In [9]:
class MatrixFactorizationEstimatorWithBias(object):
    def __init__(self,lamb=0,eps=1e-4,K=3,I = 10,doprint = False,itprint = 10 ):
        self.eps = eps
        self.K = K
        self.I = I
        self.lamb = lamb
        self.itprint = itprint
        self.doprint = doprint
    def initialize(self,R):
        self.P = np.random.rand(R.shape[0],self.K)#utilisateurs
        self.Q = np.random.rand(R.shape[1],self.K).T #items
        self.log_loss=[]
        self.log_score=[]
    def fit(self, R):
        self.initialize(R)
       
        #calcule biais initiaux
        self.mu = R[np.nonzero(R)].mean()
        self.B_u = np.array([R[i,:][np.nonzero(R[i,:])].mean() for i in range(R.shape[0])]) - self.mu
        self.B_i= np.array([R[:,j][np.nonzero(R[:,j])].mean() for j in range(R.shape[1])]) - self.mu
       
        ind = np.nonzero(R)
        nb_ex = len(ind[0])
        for it in xrange(self.I): 
            #recupere les indices des exemples
            loss= 0.
            for stoch in xrange(nb_ex):#gradient stochastique
                rand_ind = np.random.randint(nb_ex)
                i, j = ind[0][rand_ind], ind[1][rand_ind]
               
                rij = self.mu + self.B_u[i] + self.B_i[j] + np.dot(self.P[i,:], self.Q[:,j])
                eij = R[i,j] - rij               
               
                temp_P = (1 - self.lamb * self.eps) * self.P[i,:] + 2 * self.eps * eij * self.Q[:,j]
                self.Q[:,j] = (1 - self.lamb * self.eps) * self.Q[:,j] + 2 * self.eps * eij * self.P[i,:]
                self.P[i,:] = temp_P#maj simultanée
   
                self.B_u[i] = (1 - self.lamb * self.eps) * self.B_u[i] + 2 * self.eps * eij
                self.B_i[j] = (1 - self.lamb * self.eps) * self.B_i[j] + 2 * self.eps * eij
                self.mu = (1 - self.lamb * self.eps) * self.mu + 2 * self.eps * eij
               
                loss += eij**2
           
            self.log_loss.append(loss/nb_ex)
            if it % self.itprint==0:
                if self.doprint :
                    #print '\nit = ', it
                    print 'loss : ', np.mean(self.log_loss[:-self.itprint])
                    score = self.score(R)
                    self.log_score.append(score)
                    print 'score ',score
                    print 'true loss', self.loss(R)
  
    def predict(self, test_m):       
        return ((self.mu + np.dot(self.P, self.Q)).T + self.B_u).T  + self.B_i
    def score(self, m):
        return ((m[np.nonzero(m)] - self.predict(m)[np.nonzero(m)] )**2).mean()
    def loss(self, R):
        return np.power(R[np.nonzero(R)] - self.predict(R)[np.nonzero(R)], 2).sum() + self.lamb*((self.P**2).sum() + (self.Q**2).sum() + (self.B_u**2).sum() + (self.B_i**2).sum())


In [10]:
estimateur2 = MatrixFactorizationEstimatorWithBias(lamb=0.2,eps=1e-5,K=10,I = 1000,doprint = True,itprint = 50)
estimateur2.fit(train_m)
score_train = estimateur2.score(train_m)
print 'score BIAS, train : ',score_train
score_test = estimateur2.score(test_m)
print 'score BIAS, test : ',score_test

loss : nan
score  1.44356364286
true loss 117153.620756
loss :  2.94672604467
score  1.07754102464
true loss 87932.1923028
loss :  1.21396883783
score  0.994377206742
true loss 81280.6899857
loss :  1.12263422749
score  0.953114539313
true loss 77975.1265255
loss :  1.07267448648
score  0.926705224171
true loss 75856.5192978
loss :  1.03945525739
score  0.908775342635
true loss 74415.6348851
loss :  1.0154365205
score  0.89773566825
true loss 73525.2919627
loss :  0.996997432402
score  0.887660103129
true loss 72712.6812783
loss :  0.982119739726
score  0.878843870663
true loss 72001.1312512
loss :  0.969806797434
score  0.87283657827
true loss 71514.0953908
loss :  0.959498349956
score  0.867133261049
true loss 71051.7586772
loss :  0.950417205275
score  0.862585000062
true loss 70681.8937087
loss :  0.942580958629
score  0.859542186122
true loss 70432.4233881
loss :  0.935670097367
score  0.855036277234
true loss 70066.5462117
loss :  0.929511245533
score  0.851079545799
true loss 69

/home/magma/bin/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/home/magma/bin/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:71: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Avec 200 époques de moins, l'algorithme avec biais permet de converger plus rapidement à une meilleure solution.

Par contre le surapprentissage est toujours marqué, le modèle plus complexe permet donc de plus coller aux données de manières dangereuses. La généralisation n'est pas forcément évidente.

Il faudrait conduire une étude expérimentale plus poussée pour déterminer les meilleures valeurs de coefficients de régularisation ainsi que des pas de gradients.

(mais les calculs sont assez coûteux)



On sauve le modèle pour la visualisation de données avec t-SNE

In [ ]:
np.save('P_bias',estimateur2.P)
np.save('Q_bias',estimateur2.Q)